In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_predict, cross_val_score, train_test_split, ParameterSampler, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score, confusion_matrix, ConfusionMatrixDisplay, plot_roc_curve
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler


In [ ]:
df = pd.read_csv("/Users/ardrianwong/Desktop/Columbia/Fall2022/coms4995AppliedML/AML_Project/diabetes_binary_health_indicators_BRFSS2015.csv")

In [ ]:
df.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [ ]:
df.groupby(['HighBP', 'Diabetes_binary']).size().unstack()

In [ ]:
df.groupby(['HighBP', 'Diabetes_binary']).size().unstack().rename(index={0.0: 'No', 1.0: 'Yes'}, columns={0.0: 'Non-diabetic', 1.0: 'Diabetic'},)
#.set_levels(['c1','f1'],level=1,inplace=True)
# rename(index={0.0: 'Non-diabetic', 1.0: 'Diabetic'})

In [ ]:
df.groupby(['HighBP', 'Diabetes_binary']).size().unstack().rename(index={0.0: 'No', 1.0: 'Yes'}, columns={0.0: 'Non-diabetic', 1.0: 'Diabetic'}).plot(kind='bar', stacked=True, ylabel='count')


In [ ]:
# df2 = df.groupby(['Name', 'Abuse/NFF'])['Name'].count().unstack('Abuse/NFF').fillna(0)
# df2[['abuse','nff']].plot(kind='bar', stacked=True)
# df.pivot_table(index = 'HighBP', columns = 'Diabetes_binary' , values = 'counts').plot(kind = 'bar', stacked = True)
df.groupby(['Diabetes_binary', 'HighBP']).size().unstack().rename(index={0.0: 'Non-diabetic', 1.0: 'Diabetic'}).plot(kind='bar', stacked=True, ylabel='count')
df.groupby(['HighChol', 'Diabetes_binary']).size().unstack().plot(kind='bar', stacked=True, ylabel='count')
# df.plot()

In [ ]:
# MentHlth
# PhysHlth
# Age
# Income
# BMI
# Diabetes_binary
columns= df.drop(['MentHlth', 'PhysHlth', 'Age', 'Income', 'BMI', 'Diabetes_binary'], axis=1).columns
fig, axes = plt.subplots(4, 4, figsize=(40,30))
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['axes.titlesize'] = 15

i = 0
for r in range(4):
    for c in range(4):
        df.groupby(['Diabetes_binary', columns[i]]).size().unstack().rename(index={0.0: 'Non-diabetic', 1.0: 'Diabetic'}).plot(kind='bar', rot = 0, fontsize=15, stacked=True, title=f'{columns[i]} vs Diabetes', ylabel='count', ax=axes[r,c])
        i+=1


In [ ]:
df.drop(['MentHlth', 'PhysHlth', 'Age', 'Income', 'BMI', 'Diabetes_binary'], axis=1).columns


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_predict, cross_val_score, train_test_split, ParameterSampler, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score, confusion_matrix, ConfusionMatrixDisplay, plot_roc_curve
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from scipy.stats import uniform


In [ ]:
len(df) - len(df.drop_duplicates())

In [ ]:
df.drop_duplicates()

In [ ]:
non_dup_df = df.drop_duplicates()
y = non_dup_df['Diabetes_binary']
X = non_dup_df.drop('Diabetes_binary', axis=1)

In [ ]:
len(df) - 24206

229474

In [ ]:
# 2. Split the dataset into training (60%), validation (20%), and test (20%) sets
X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# X_train, X_val, y_train, y_val = train_test_split(X_dev, y_dev, test_size=0.25, random_state=0)

# 3. Standardize the columns in the feature matrices
scaler = StandardScaler()
X_dev = scaler.fit_transform(X_dev)   # Fit and transform scalar on X_train
X_test = scaler.transform(X_test)         # Transform X_test
print(len(X_dev), len(X_test), str(len(X_dev) + len(X_test)))
sm = SMOTE(random_state=0)
X_dev_sm, y_dev_sm = sm.fit_resample(X_dev, y_dev)
print(len(X_dev_sm))

183579 45895 229474
311006


In [ ]:
len(y_dev_sm)

311006

In [ ]:
from sklearn.metrics import accuracy_score
# y_pred = [0, 2, 1, 3]
# y_true = [0, 1, 2, 3]
# accuracy_score(y_true, y_pred)

In [ ]:
#SMOTE
SVM = LinearSVC(max_iter=5000, dual=False, random_state=0)
distributions = dict(C=uniform(loc=0, scale=10))
clf = RandomizedSearchCV(SVM, distributions, cv=5,
                         scoring =['recall', 'f1', 'precision', 'accuracy'],
                         refit='recall',
                         verbose=3,
                         random_state=0)
search1 = clf.fit(X_dev_sm, y_dev_sm)
search1.best_params_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END C=5.4881350392732475; accuracy: (test=0.734) f1: (test=0.740) precision: (test=0.723) recall: (test=0.758) total time=   0.7s
[CV 2/5] END C=5.4881350392732475; accuracy: (test=0.738) f1: (test=0.747) precision: (test=0.722) recall: (test=0.773) total time=   0.7s
[CV 3/5] END C=5.4881350392732475; accuracy: (test=0.740) f1: (test=0.748) precision: (test=0.726) recall: (test=0.772) total time=   0.7s
[CV 4/5] END C=5.4881350392732475; accuracy: (test=0.739) f1: (test=0.749) precision: (test=0.722) recall: (test=0.779) total time=   0.7s
[CV 5/5] END C=5.4881350392732475; accuracy: (test=0.742) f1: (test=0.750) precision: (test=0.728) recall: (test=0.773) total time=   0.7s
[CV 1/5] END C=7.151893663724195; accuracy: (test=0.734) f1: (test=0.740) precision: (test=0.723) recall: (test=0.758) total time=   0.6s
[CV 2/5] END C=7.151893663724195; accuracy: (test=0.738) f1: (test=0.747) precision: (test=0.722) recall: 

{'C': 5.4881350392732475}

In [ ]:
accuracy_score(search1.best_estimator_.predict(X_test), y_test)

0.7126266477829829

In [ ]:
SVM2 = LinearSVC(max_iter=1000, dual=False, random_state=0)
distributions2 = dict(C=uniform(loc=0, scale=10))
clf2 = RandomizedSearchCV(SVM2, distributions, cv=5,
                         scoring =['recall', 'f1', 'precision', 'accuracy'],
                         refit='recall',
                         verbose=3,
                         random_state=0)
search2 = clf2.fit(X_dev, y_dev)
search2.best_params_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END C=5.4881350392732475; accuracy: (test=0.850) f1: (test=0.126) precision: (test=0.590) recall: (test=0.070) total time=   0.5s
[CV 2/5] END C=5.4881350392732475; accuracy: (test=0.850) f1: (test=0.117) precision: (test=0.571) recall: (test=0.065) total time=   0.5s
[CV 3/5] END C=5.4881350392732475; accuracy: (test=0.850) f1: (test=0.114) precision: (test=0.594) recall: (test=0.063) total time=   0.5s
[CV 4/5] END C=5.4881350392732475; accuracy: (test=0.850) f1: (test=0.121) precision: (test=0.580) recall: (test=0.068) total time=   0.4s
[CV 5/5] END C=5.4881350392732475; accuracy: (test=0.851) f1: (test=0.118) precision: (test=0.612) recall: (test=0.066) total time=   0.4s
[CV 1/5] END C=7.151893663724195; accuracy: (test=0.850) f1: (test=0.126) precision: (test=0.590) recall: (test=0.070) total time=   0.4s
[CV 2/5] END C=7.151893663724195; accuracy: (test=0.850) f1: (test=0.117) precision: (test=0.571) recall: 

{'C': 5.4881350392732475}

In [ ]:
# accuracy_score(search2.best_estimator_.predict(X_test), y_test)
search2.best_estimator_.score(X_test, y_test)

0.8498311362893561

In [ ]:
#Default, no smote
SVM3 = LinearSVC(max_iter=1000, penalty='l2', dual=False, random_state=0)
distributions3 = dict(C=uniform(loc=0, scale=10))
clf3 = RandomizedSearchCV(SVM3, distributions3, cv=5,
                         scoring =['recall', 'f1', 'precision', 'accuracy'],
                         refit='recall',
                         verbose=3,
                         n_iter=100,
                         random_state=0)
search3 = clf3.fit(X_dev, y_dev)
search3.best_params_

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 1/5] END C=5.4881350392732475; accuracy: (test=0.850) f1: (test=0.126) precision: (test=0.590) recall: (test=0.070) total time=   0.5s
[CV 2/5] END C=5.4881350392732475; accuracy: (test=0.850) f1: (test=0.117) precision: (test=0.571) recall: (test=0.065) total time=   0.5s
[CV 3/5] END C=5.4881350392732475; accuracy: (test=0.850) f1: (test=0.114) precision: (test=0.594) recall: (test=0.063) total time=   0.5s
[CV 4/5] END C=5.4881350392732475; accuracy: (test=0.850) f1: (test=0.121) precision: (test=0.580) recall: (test=0.068) total time=   0.4s
[CV 5/5] END C=5.4881350392732475; accuracy: (test=0.851) f1: (test=0.118) precision: (test=0.612) recall: (test=0.066) total time=   0.5s
[CV 1/5] END C=7.151893663724195; accuracy: (test=0.850) f1: (test=0.126) precision: (test=0.590) recall: (test=0.070) total time=   0.4s
[CV 2/5] END C=7.151893663724195; accuracy: (test=0.850) f1: (test=0.117) precision: (test=0.571) recall

[CV 5/5] END C=5.288949197529044; accuracy: (test=0.851) f1: (test=0.118) precision: (test=0.612) recall: (test=0.066) total time=   0.4s
[CV 1/5] END C=5.680445610939323; accuracy: (test=0.850) f1: (test=0.126) precision: (test=0.590) recall: (test=0.070) total time=   0.5s
[CV 2/5] END C=5.680445610939323; accuracy: (test=0.850) f1: (test=0.117) precision: (test=0.571) recall: (test=0.065) total time=   0.4s
[CV 3/5] END C=5.680445610939323; accuracy: (test=0.850) f1: (test=0.114) precision: (test=0.594) recall: (test=0.063) total time=   0.4s
[CV 4/5] END C=5.680445610939323; accuracy: (test=0.850) f1: (test=0.121) precision: (test=0.580) recall: (test=0.068) total time=   0.4s
[CV 5/5] END C=5.680445610939323; accuracy: (test=0.851) f1: (test=0.118) precision: (test=0.612) recall: (test=0.066) total time=   0.4s
[CV 1/5] END C=9.25596638292661; accuracy: (test=0.850) f1: (test=0.126) precision: (test=0.590) recall: (test=0.070) total time=   0.4s
[CV 2/5] END C=9.25596638292661; ac

[CV 5/5] END C=7.805291762864554; accuracy: (test=0.851) f1: (test=0.118) precision: (test=0.612) recall: (test=0.066) total time=   0.4s
[CV 1/5] END C=1.1827442586893322; accuracy: (test=0.850) f1: (test=0.126) precision: (test=0.590) recall: (test=0.070) total time=   0.4s
[CV 2/5] END C=1.1827442586893322; accuracy: (test=0.850) f1: (test=0.117) precision: (test=0.571) recall: (test=0.065) total time=   0.5s
[CV 3/5] END C=1.1827442586893322; accuracy: (test=0.850) f1: (test=0.114) precision: (test=0.594) recall: (test=0.063) total time=   0.5s
[CV 4/5] END C=1.1827442586893322; accuracy: (test=0.850) f1: (test=0.121) precision: (test=0.580) recall: (test=0.068) total time=   0.4s
[CV 5/5] END C=1.1827442586893322; accuracy: (test=0.851) f1: (test=0.118) precision: (test=0.612) recall: (test=0.066) total time=   0.4s
[CV 1/5] END C=6.399210213275238; accuracy: (test=0.850) f1: (test=0.126) precision: (test=0.590) recall: (test=0.070) total time=   0.4s
[CV 2/5] END C=6.399210213275

[CV 5/5] END C=6.176354970758771; accuracy: (test=0.851) f1: (test=0.118) precision: (test=0.612) recall: (test=0.066) total time=   0.4s
[CV 1/5] END C=6.120957227224214; accuracy: (test=0.850) f1: (test=0.126) precision: (test=0.590) recall: (test=0.070) total time=   0.4s
[CV 2/5] END C=6.120957227224214; accuracy: (test=0.850) f1: (test=0.117) precision: (test=0.571) recall: (test=0.065) total time=   0.4s
[CV 3/5] END C=6.120957227224214; accuracy: (test=0.850) f1: (test=0.114) precision: (test=0.594) recall: (test=0.063) total time=   0.4s
[CV 4/5] END C=6.120957227224214; accuracy: (test=0.850) f1: (test=0.121) precision: (test=0.580) recall: (test=0.068) total time=   0.4s
[CV 5/5] END C=6.120957227224214; accuracy: (test=0.851) f1: (test=0.118) precision: (test=0.612) recall: (test=0.066) total time=   0.4s
[CV 1/5] END C=6.169339968747569; accuracy: (test=0.850) f1: (test=0.126) precision: (test=0.590) recall: (test=0.070) total time=   0.4s
[CV 2/5] END C=6.169339968747569; 

[CV 5/5] END C=1.289262976548533; accuracy: (test=0.851) f1: (test=0.118) precision: (test=0.612) recall: (test=0.066) total time=   0.4s
[CV 1/5] END C=3.1542835092418384; accuracy: (test=0.850) f1: (test=0.126) precision: (test=0.590) recall: (test=0.070) total time=   0.4s
[CV 2/5] END C=3.1542835092418384; accuracy: (test=0.850) f1: (test=0.117) precision: (test=0.571) recall: (test=0.065) total time=   0.4s
[CV 3/5] END C=3.1542835092418384; accuracy: (test=0.850) f1: (test=0.114) precision: (test=0.594) recall: (test=0.063) total time=   0.4s
[CV 4/5] END C=3.1542835092418384; accuracy: (test=0.850) f1: (test=0.121) precision: (test=0.580) recall: (test=0.068) total time=   0.4s
[CV 5/5] END C=3.1542835092418384; accuracy: (test=0.851) f1: (test=0.118) precision: (test=0.612) recall: (test=0.066) total time=   0.4s
[CV 1/5] END C=3.637107709426226; accuracy: (test=0.850) f1: (test=0.126) precision: (test=0.590) recall: (test=0.070) total time=   0.4s
[CV 2/5] END C=3.637107709426

[CV 5/5] END C=2.4442559200160274; accuracy: (test=0.851) f1: (test=0.118) precision: (test=0.612) recall: (test=0.066) total time=   0.4s
[CV 1/5] END C=1.5896958364551972; accuracy: (test=0.850) f1: (test=0.126) precision: (test=0.590) recall: (test=0.070) total time=   0.4s
[CV 2/5] END C=1.5896958364551972; accuracy: (test=0.850) f1: (test=0.117) precision: (test=0.571) recall: (test=0.065) total time=   0.4s
[CV 3/5] END C=1.5896958364551972; accuracy: (test=0.850) f1: (test=0.114) precision: (test=0.594) recall: (test=0.063) total time=   0.4s
[CV 4/5] END C=1.5896958364551972; accuracy: (test=0.850) f1: (test=0.121) precision: (test=0.580) recall: (test=0.068) total time=   0.4s
[CV 5/5] END C=1.5896958364551972; accuracy: (test=0.851) f1: (test=0.118) precision: (test=0.612) recall: (test=0.066) total time=   0.4s
[CV 1/5] END C=1.1037514116430513; accuracy: (test=0.850) f1: (test=0.126) precision: (test=0.590) recall: (test=0.070) total time=   0.4s
[CV 2/5] END C=1.1037514116

[CV 5/5] END C=4.686512016477016; accuracy: (test=0.851) f1: (test=0.118) precision: (test=0.612) recall: (test=0.066) total time=   0.4s
[CV 1/5] END C=9.767610881903371; accuracy: (test=0.850) f1: (test=0.126) precision: (test=0.590) recall: (test=0.070) total time=   0.4s
[CV 2/5] END C=9.767610881903371; accuracy: (test=0.850) f1: (test=0.117) precision: (test=0.571) recall: (test=0.065) total time=   0.4s
[CV 3/5] END C=9.767610881903371; accuracy: (test=0.850) f1: (test=0.114) precision: (test=0.594) recall: (test=0.063) total time=   0.4s
[CV 4/5] END C=9.767610881903371; accuracy: (test=0.850) f1: (test=0.121) precision: (test=0.580) recall: (test=0.068) total time=   0.4s
[CV 5/5] END C=9.767610881903371; accuracy: (test=0.851) f1: (test=0.118) precision: (test=0.612) recall: (test=0.066) total time=   0.4s
[CV 1/5] END C=6.04845519745046; accuracy: (test=0.850) f1: (test=0.126) precision: (test=0.590) recall: (test=0.070) total time=   0.5s
[CV 2/5] END C=6.04845519745046; ac

[CV 5/5] END C=6.924721193700199; accuracy: (test=0.851) f1: (test=0.118) precision: (test=0.612) recall: (test=0.066) total time=   0.4s
[CV 1/5] END C=5.666014542065751; accuracy: (test=0.850) f1: (test=0.126) precision: (test=0.590) recall: (test=0.070) total time=   0.4s
[CV 2/5] END C=5.666014542065751; accuracy: (test=0.850) f1: (test=0.117) precision: (test=0.571) recall: (test=0.065) total time=   0.4s
[CV 3/5] END C=5.666014542065751; accuracy: (test=0.850) f1: (test=0.114) precision: (test=0.594) recall: (test=0.063) total time=   0.4s
[CV 4/5] END C=5.666014542065751; accuracy: (test=0.850) f1: (test=0.121) precision: (test=0.580) recall: (test=0.068) total time=   0.4s
[CV 5/5] END C=5.666014542065751; accuracy: (test=0.851) f1: (test=0.118) precision: (test=0.612) recall: (test=0.066) total time=   0.4s
[CV 1/5] END C=2.653894909394454; accuracy: (test=0.850) f1: (test=0.126) precision: (test=0.590) recall: (test=0.070) total time=   0.4s
[CV 2/5] END C=2.653894909394454; 

[CV 5/5] END C=1.8319136200711683; accuracy: (test=0.851) f1: (test=0.118) precision: (test=0.612) recall: (test=0.066) total time=   0.4s
[CV 1/5] END C=5.865129348100831; accuracy: (test=0.850) f1: (test=0.126) precision: (test=0.590) recall: (test=0.070) total time=   0.4s
[CV 2/5] END C=5.865129348100831; accuracy: (test=0.850) f1: (test=0.117) precision: (test=0.571) recall: (test=0.065) total time=   0.5s
[CV 3/5] END C=5.865129348100831; accuracy: (test=0.850) f1: (test=0.114) precision: (test=0.594) recall: (test=0.063) total time=   0.4s
[CV 4/5] END C=5.865129348100831; accuracy: (test=0.850) f1: (test=0.121) precision: (test=0.580) recall: (test=0.068) total time=   0.4s
[CV 5/5] END C=5.865129348100831; accuracy: (test=0.851) f1: (test=0.118) precision: (test=0.612) recall: (test=0.066) total time=   0.4s
[CV 1/5] END C=0.20107546187493552; accuracy: (test=0.850) f1: (test=0.126) precision: (test=0.590) recall: (test=0.070) total time=   0.4s
[CV 2/5] END C=0.20107546187493

{'C': 5.4881350392732475}

In [ ]:
accuracy_score(search3.best_estimator_.predict(X_test), y_test)

0.8498311362893561

In [ ]:
#5 non smote, 5 class balance, 5 with smote

In [ ]:
search.best_score_

In [ ]:
dir(search)

In [ ]:
search.best_estimator_.score

In [ ]:
dir(search.best_estimator_)